This code is optimized to count inclusions and nuclei for images with transfections and cotransfections.

Import Libraries

In [4]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.filters import gaussian, threshold_otsu
from skimage import morphology, exposure
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_dilation, disk

Define Sub Functions

In [5]:
# Function to extract image file paths from a specified folder
def extract_image_paths(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Function to preprocess and segment the DAPI channel to count nuclei
def count_nuclei(dapi_channel):
    # Apply Gaussian blur to smooth out noise
    blurred_dapi = gaussian(dapi_channel, sigma=2)
    
    # Thresholding to segment nuclei
    threshold_value = threshold_otsu(blurred_dapi)
    binary_image = blurred_dapi > threshold_value
    
    # Morphological operations to clean up the segmentation
    filled_image = morphology.remove_small_holes(binary_image, area_threshold=50000)
    cleaned_image = remove_small_objects(filled_image, min_size=400)
    
    # Dilate to merge adjacent objects
    selem = disk(5)
    merged_image = binary_dilation(cleaned_image, footprint=selem)
    
    # Label the objects and count the nuclei
    labeled_image = label(merged_image)
    n_objects = len(np.unique(labeled_image)) - 1  # Exclude background label (0)
    
    return n_objects, labeled_image

# Function to preprocess and segment the green channel to count inclusions
def count_inclusions(green_channel):
    # Adjust brightness and contrast
    adjusted_img = exposure.adjust_sigmoid(green_channel, cutoff=0.4)
    
    # Normalize the pixel values
    normalized_img = (adjusted_img - adjusted_img.min()) / (adjusted_img.max() - adjusted_img.min())
    
    # Thresholding to segment inclusions
    threshold_value = 0.3  # Adjust as needed
    binary_image = normalized_img > threshold_value
    
    # Label the inclusions and measure their sizes
    labeled_image = label(binary_image)
    props = regionprops(labeled_image, normalized_img)
    
    sizes = [prop.area for prop in props]
    
    # Filter out very small inclusions
    filtered_sizes = [size for size in sizes if size > 10]
    
    return filtered_sizes, labeled_image

# Function to analyze all images in a folder
def analyze_images(folder_path):
    # Extract all image file paths
    images_to_analyze = extract_image_paths(folder_path)
    
    
    # Initialize lists to store results
    number_of_nuclei_list = []
    mean_sizes_of_inclusions = []
    
    sizes_df_new = pd.DataFrame()
    
    # Iterate through each image
    for iteration, image_path in enumerate(images_to_analyze, start=1):
        # Read the image
        image = imread(image_path)
        
        # Extract channels
        dapi_channel = image[0]  # DAPI channel
        green_channel = image[1]  # Green channel
        
        # Count nuclei
        n_nuclei, labeled_nuclei_image = count_nuclei(dapi_channel)
        number_of_nuclei_list.append(n_nuclei)
        
        # Count inclusions
        inclusion_sizes, labeled_inclusion_image = count_inclusions(green_channel)
        mean_sizes_of_inclusions.append(np.mean(inclusion_sizes))
        
        # Create DataFrame for current image's inclusion sizes
        sizes_df_add = pd.DataFrame(inclusion_sizes, columns=[f'image {iteration}'])
        
        # Concatenate to the main DataFrame
        sizes_df_new = pd.concat([sizes_df_new, sizes_df_add], axis=1)
    
    return sizes_df_new, number_of_nuclei_list, mean_sizes_of_inclusions, images_to_analyze

# Function to finalize and save the results to Excel files
def save_results(sizes_df_new, number_of_nuclei_list, mean_sizes_of_inclusions, images_to_analyze):
    basenames = [os.path.basename(path) for path in images_to_analyze]
    # Replace all values of 1 with NaN and drop NaN values to the bottom
    sizes_df_new = sizes_df_new.replace(1.0, np.NaN)
    sizes_df_new = sizes_df_new.apply(lambda x: pd.Series(x.dropna().values))
    
    # Transpose for nuclei count
    sizes_df_new_nuclei = sizes_df_new.transpose()
    
    # Count total number of inclusions per image
    number_of_inclusions = sizes_df_new_nuclei.count(axis=1)
    
    # Create DataFrame for the second Excel file
    excel_2 = pd.DataFrame({
        'Image_Name': basenames,
        'Number_of_Inclusions': number_of_inclusions,
        'Number_of_Nuclei': number_of_nuclei_list,
        'Average_Number_of_Inclusions_per_Cell': number_of_inclusions / number_of_nuclei_list,
        'Mean_Sizes_Of_Inclusions': mean_sizes_of_inclusions
    })
    
    # Replace NaN values with empty strings
    sizes_df_new = sizes_df_new.fillna('')
    
    # Save DataFrames to Excel files
    # sizes_df_new.to_excel("output.xlsx")
    excel_2.to_excel("SUMMARY.xlsx")

Define Main Function

In [6]:
# Main function to run the analysis
def main():
    folder_path = "test_images_sirnas"
    sizes_df_new, number_of_nuclei_list, mean_sizes_of_inclusions, images_to_analyze = analyze_images(folder_path)
    save_results(sizes_df_new, number_of_nuclei_list, mean_sizes_of_inclusions, images_to_analyze)

# Run the main function
if __name__ == "__main__":
    main()